In [ ]:
!pip install openai
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
import openai
from getpass import getpass
import os
import pandas as pd

In [ ]:
OPENAI_API_KEY = getpass("Enter your OpenAI API key: ")
openai.api_key = OPENAI_API_KEY

Enter your OpenAI API key: ··········


#### Cargar Dataset

In [ ]:
df = pd.read_csv('generic-food.csv')
df.head(10)


,FOOD NAME,SCIENTIFIC NAME,GROUP,SUB GROUP
0,Angelica,Angelica keiskei,Herbs and Spices,Herbs
1,Savoy cabbage,Brassica oleracea var. sabauda,Vegetables,Cabbages
2,Silver linden,Tilia argentea,Herbs and Spices,Herbs
3,Kiwi,Actinidia chinensis,Fruits,Tropical fruits
4,Allium (Onion),Allium,Vegetables,Onion-family vegetables
5,Garden onion,Allium cepa,Vegetables,Onion-family vegetables
6,Leek,Allium porrum,Vegetables,Onion-family vegetables
7,Garlic,Allium sativum,Herbs and Spices,Herbs
8,Chives,Allium schoenoprasum,Herbs and Spices,Herbs
9,Lemon verbena,Aloysia triphylla,Herbs and Spices,Herbs


In [ ]:
import tiktoken

def num_tokens_from_string(text, encoding_name):
  encoding = tiktoken.get_encoding(encoding_name)
  num_tokens = len(encoding.encode(text))
  return num_tokens


In [ ]:
df['total_tokens'] = df['FOOD NAME'].apply(lambda x : num_tokens_from_string(x, 'cl100k_base'))

In [ ]:
df.head(10)

,FOOD NAME,SCIENTIFIC NAME,GROUP,SUB GROUP,total_tokens
0,Angelica,Angelica keiskei,Herbs and Spices,Herbs,2
1,Savoy cabbage,Brassica oleracea var. sabauda,Vegetables,Cabbages,4
2,Silver linden,Tilia argentea,Herbs and Spices,Herbs,3
3,Kiwi,Actinidia chinensis,Fruits,Tropical fruits,2
4,Allium (Onion),Allium,Vegetables,Onion-family vegetables,6
5,Garden onion,Allium cepa,Vegetables,Onion-family vegetables,3
6,Leek,Allium porrum,Vegetables,Onion-family vegetables,2
7,Garlic,Allium sativum,Herbs and Spices,Herbs,2
8,Chives,Allium schoenoprasum,Herbs and Spices,Herbs,2
9,Lemon verbena,Aloysia triphylla,Herbs and Spices,Herbs,4


In [ ]:
sum(df['total_tokens'])


2947

#### Generando embeddings

In [ ]:
def get_embedding(text, model='text-embedding-ada-002'):
  text = text.replace("\n", " ")
  return openai.Embedding.create(input=[text], model=model)['data'][0]['embedding']

In [ ]:
df['ada_embeddings'] = df['FOOD NAME'].apply(lambda x : get_embedding(x, model='text-embedding-ada-002'))


In [ ]:
df.head()

,FOOD NAME,SCIENTIFIC NAME,GROUP,SUB GROUP,total_tokens,ada_embeddings
0,Angelica,Angelica keiskei,Herbs and Spices,Herbs,2,"[0.0062030586414039135, -0.010099552571773529,..."
1,Savoy cabbage,Brassica oleracea var. sabauda,Vegetables,Cabbages,4,"[0.005501243751496077, -0.004981592763215303, ..."
2,Silver linden,Tilia argentea,Herbs and Spices,Herbs,3,"[-0.004518528468906879, 0.01999432034790516, -..."
3,Kiwi,Actinidia chinensis,Fruits,Tropical fruits,2,"[-0.004570240620523691, -0.010097870603203773,..."
4,Allium (Onion),Allium,Vegetables,Onion-family vegetables,6,"[0.013196941465139389, -0.019988618791103363, ..."


In [ ]:
embedding_prueba = get_embedding('esto es una prueba de embeddings para openAI')

In [ ]:
len(embedding_prueba)

1536

#### Creando datasets para visualizar

In [ ]:
df_embeddings = pd.DataFrame(list(df['ada_embeddings']))

In [ ]:
df_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,0.006203,-0.010100,0.004670,-0.028905,-0.005343,0.013059,-0.016319,0.007246,-0.010026,-0.028185,...,0.024226,-0.011499,0.004516,-0.022692,-0.008846,0.030932,0.008340,-0.003983,-0.022479,-0.015106
1,0.005501,-0.004982,0.014948,-0.010938,-0.002452,0.021107,-0.008905,-0.024327,-0.018053,-0.039699,...,0.017925,-0.019824,0.026791,-0.032693,-0.019375,0.020016,-0.025585,0.010464,-0.007686,0.000767
2,-0.004519,0.019994,-0.002004,-0.024117,-0.016691,0.020505,-0.041278,-0.001932,-0.001280,-0.035477,...,0.002937,-0.022358,0.038538,-0.026346,-0.032469,0.021109,-0.000254,-0.020505,-0.011582,-0.033409
3,-0.004570,-0.010098,-0.009199,-0.010053,-0.007821,0.023016,-0.014322,-0.016250,-0.001695,-0.036484,...,0.030947,-0.012110,0.022512,-0.011799,-0.005340,0.010661,-0.021774,-0.001690,-0.014063,-0.020713
4,0.013197,-0.019989,0.002220,-0.010073,-0.018574,0.022228,-0.018011,-0.015286,-0.008887,-0.020591,...,0.020198,-0.034476,0.031568,-0.011049,-0.019373,0.000330,0.010328,0.001156,-0.001222,0.012431


In [ ]:
df_embeddings.to_csv('embedding_food.tsv', sep='\t', index=False, header=False)

In [ ]:
df[['FOOD NAME', 'GROUP', 'SUB GROUP']].to_csv('labels_food.tsv', sep='\t', index=False, header=True)
